# ⚡ Spectral Affinity: Harmonic Set Generator

This notebook analyzes your library and automatically generates **Harmonic DJ Sets** of 50-60 minutes each.

**Features:**
- **🧠 Essentia AI:** High-precision Key & BPM detection.
- **⏱️ Auto-Clustering:** Groups tracks into ready-to-mix playlists (~60 mins).
- **🔄 Harmonic Flow:** Tracks inside each set are ordered by the Circle of Fifths for perfect mixing.
- **📦 Auto-Download:** Automatically zips and triggers download of your sets.

---

### 1. 🛠️ Setup & Acceleration Engines

In [ ]:
import os
import warnings
import sys

# Suppress warnings
warnings.filterwarnings('ignore')

# 📦 Install Essentia (High-Performance Audio AI)
try:
    import essentia
except ImportError:
    print("⚙️ Installing Essentia AI acceleration...")
    !pip install -q essentia joblib tqdm

import essentia.standard as es
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import shutil
import glob
import re
import numpy as np
from IPython.display import HTML, FileLink, display

print(f"✅ System Ready. Accelerators: {os.cpu_count()} cores active.")

### 2. 🧠 The Analyzer Engine (Key + BPM + Time)

In [ ]:
def get_camelot(key, scale):
    # Standard Camelot Mixing Wheel Map
    camelot_map = {
        'B': {'major': '01B', 'minor': '10A'},
        'F#': {'major': '02B', 'minor': '11A'}, 'Gb': {'major': '02B', 'minor': '11A'},
        'C#': {'major': '03B', 'minor': '12A'}, 'Db': {'major': '03B', 'minor': '12A'},
        'G#': {'major': '04B', 'minor': '01A'}, 'Ab': {'major': '04B', 'minor': '01A'},
        'D#': {'major': '05B', 'minor': '02A'}, 'Eb': {'major': '05B', 'minor': '02A'},
        'A#': {'major': '06B', 'minor': '03A'}, 'Bb': {'major': '06B', 'minor': '03A'},
        'F': {'major': '07B', 'minor': '04A'},
        'C': {'major': '08B', 'minor': '05A'},
        'G': {'major': '09B', 'minor': '06A'},
        'D': {'major': '10B', 'minor': '07A'},
        'A': {'major': '11B', 'minor': '08A'},
        'E': {'major': '12B', 'minor': '09A'}
    }
    key = key.strip()
    scale = scale.strip().lower()
    if key in camelot_map and scale in camelot_map[key]:
        return camelot_map[key][scale]
    return "00X"

def analyze_track_ai(file_path):
    try:
        # 1. Load Audio efficiently (Resample to 44.1 for standard analysis)
        loader = es.MonoLoader(filename=file_path, sampleRate=44100)
        audio = loader()
        
        # 2. Get Duration
        duration_sec = len(audio) / 44100.0

        # 3. Key Detection (HPCP)
        key_extractor = es.KeyExtractor(averageDetuning=True, profileType='edma')
        key, scale, strength = key_extractor(audio)

        # 4. BPM Detection
        rhythm_extractor = es.RhythmExtractor2013(method="multifeature")
        bpm, _, _, _, _ = rhythm_extractor(audio)

        # 5. Camelot Code
        camelot = get_camelot(key, scale)
        
        return {
            "path": file_path,
            "key": key,
            "scale": scale,
            "bpm": round(bpm, 1),
            "duration": duration_sec,
            "camelot": camelot,
            "valid": True
        }
    except Exception as e:
        return {"path": file_path, "valid": False}

def clean_filename(filename):
    if '.' not in filename: return filename
    name_body, ext = filename.rsplit('.', 1)
    patterns = [
        r"^Slavic-", r"^Theme_OST-", r"^My_Workspace-", r"^audio-",
        r"[\(\.\-_\s]?[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}.*?$"
    ]
    for p in patterns: name_body = re.sub(p, "", name_body, flags=re.IGNORECASE)
    name_body = name_body.replace("_", " ").strip(" -(_)")
    name_body = re.sub(r"\s+", " ", name_body).strip()
    return f"{name_body if name_body else 'Unnamed'}.{ext}"

### 3. 🚀 Generator Pipeline (Sets of 50-60min)

In [ ]:
# --- CONFIGURATION ---
INPUT_DIR = "/kaggle/input/datasets/danieldobles/ost-songs"
OUTPUT_DIR = "/kaggle/working/harmonic_sets"
TARGET_SET_DURATION = 60 * 60  # 60 Minutes in seconds
# ---------------------

print("🔍 Scanning Library...")
audio_extensions = ['*.mp3', '*.wav', '*.flac', '*.m4a']
file_paths = []
for ext in audio_extensions:
    file_paths.extend(glob.glob(os.path.join(INPUT_DIR, "**", ext), recursive=True))
file_paths = list(set(file_paths))
print(f"📂 Found {len(file_paths)} tracks.")

if not file_paths:
    print("❌ No files found.")
else:
    print("\n🚀 Analyzing Audio (Key + BPM + Time)...")
    results = Parallel(n_jobs=-1, verbose=5)(
        delayed(analyze_track_ai)(p) for p in file_paths
    )
    library = [r for r in results if r['valid']]
    
    # --- SORTING STRATEGY ---
    library.sort(key=lambda x: (x['camelot'], x['bpm']))
    
    # --- CLUSTERING LOGIC ---
    print("\n🎧 Generating Sets...")
    if os.path.exists(OUTPUT_DIR): shutil.rmtree(OUTPUT_DIR)
    
    sets = []
    current_set = []
    current_duration = 0
    
    for track in library:
        current_set.append(track)
        current_duration += track['duration']
        
        # Close set if it exceeds target
        if current_duration >= TARGET_SET_DURATION:
            sets.append(current_set)
            current_set = []
            current_duration = 0
            
    # Add remaining tracks to final set
    if current_set: 
        sets.append(current_set)

    # --- EXPORT ---
    total_sets = len(sets)
    for i, s in enumerate(sets):
        set_num = i + 1
        num_tracks = len(s)
        duration_min = sum(t['duration'] for t in s) / 60
        
        folder_name = f"Set_{str(set_num).zfill(2)} ({int(duration_min)}m - {num_tracks} Tracks)"
        set_dir = os.path.join(OUTPUT_DIR, folder_name)
        os.makedirs(set_dir, exist_ok=True)
        
        print(f"💿 Created: {folder_name}")
        
        for idx, track in enumerate(s):
            original_name = os.path.basename(track['path'])
            clean = clean_filename(original_name)
            
            # Filename: 01 - [8B - 124BPM] Song.mp3
            prefix = str(idx + 1).zfill(2)
            meta = f"[{track['camelot']} - {int(track['bpm'])}BPM]"
            dest_name = f"{prefix} - {meta} {clean}"
            
            shutil.copy2(track['path'], os.path.join(set_dir, dest_name))

    print(f"\n✨ DONE! Generated {total_sets} harmonic sets.")
    
    # --- ZIP AND AUTO-DOWNLOAD ---
    zip_name = "harmonic_sets_ready_to_mix.zip"
    !zip -0 -rq {zip_name} harmonic_sets
    
    print(f"📦 Zip Created: {zip_name}")
    
    # Display Download Link & Attempt Auto-Click
    display(HTML(f"<h3>⬇️ <a href='{zip_name}' target='_blank'>Click here to Download Mixes</a></h3>"))
    display(FileLink(zip_name))
    
    # Javascript trigger
    display(HTML(f"""
    <script>
        console.log("Attempting auto-download...");
        var links = document.querySelectorAll('a');
        for (var i = 0; i < links.length; i++) {{
            if (links[i].href.includes('{zip_name}')) {{
                links[i].click();
                break;
            }}
        }}
    </script>
    """))